# Inventory Management

## Average Turnover Rate
The inventory turnover ratio is a key metric that shows how efficiently a company manages its inventory. It measures how many times, on average, inventory is sold and replaced over a specific period (usually one year). A higher turnover ratio generally indicates effective inventory management, meaning the company is selling goods quickly and not overstocking. Conversely, a lower turnover ratio can suggest excess inventory, which ties up cash and increases holding costs.



The inventory turnover ratio is calculated as:

<div align="center">

$$
\text{Inventory Turnover Ratio} = \frac{\text{Total Demand}}{\text{Average Inventory Level}}
$$

</div>


​
 
- Total Demand represents the total amount of inventory sold during the period.
- Average Inventory Level is typically calculated as the average of the inventory at the beginning and end of the period.


## Categorising our products

We can categorize our products into high, medium, and low demand groups. High-demand products require more frequent monitoring, while lower-demand items can be checked less often. For this analysis, our focus will be on high-demand products.

## Current Turnover Ratio

Let's calculate the current turnover ratio of products with high demand.

In [1]:
import pandas as pd
import numpy as np
from prophet import Prophet

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Importing plotly failed. Interactive plots will not work.


### A-items: High demand products
Top 20% of products that contribute the most to total demand/value.

In [7]:
data = pd.read_csv("Data/online_retail_clean.csv")

# Calculate total demand (quantity sold) and total sales value for each StockCode
product_demand = data.groupby('StockCode').agg({
    'Quantity': 'sum',  # Total quantity sold
    'TotalPrice': 'sum'  # Total revenue
}).reset_index()

# Rank products by total demand in descending order for ABC analysis
product_demand = product_demand.sort_values(by='Quantity', ascending=False)
product_demand['Cumulative_Demand'] = product_demand['Quantity'].cumsum()
total_demand = product_demand['Quantity'].sum()

# Rename 'Quantity' to 'TotalDemand' for clarity
product_demand.rename(columns={'Quantity': 'TotalDemand'}, inplace=True)


# Classify products based on cumulative demand percentages
product_demand['Demand_Percentage'] = product_demand['Cumulative_Demand'] / total_demand
product_demand['ABC_Category'] = pd.cut(
    product_demand['Demand_Percentage'],
    bins=[0, 0.2, 0.5, 1.0],  # Top 20%, next 30%, and remaining 50%
    labels=['A', 'B', 'C']
)

# import ace_tools as tools; tools.display_dataframe_to_user(name="ABC Analysis - Product Segmentation", dataframe=product_demand)

product_demand.head()


FileNotFoundError: [Errno 2] No such file or directory: 'Data/online_retail_clean.csv'

### Generate Synthetic Data to estimate the current inventory levels of the products

In [4]:
# Set a random seed for reproducibility
np.random.seed(42)

a_items_data = product_demand[product_demand['ABC_Category'] == 'A'][['StockCode', 'TotalDemand']]

# scale_factor = 1.1 for high demand products to reduce the risk of stockouts
def generate_inventory_levels(product_data, scale_factor=1.1, base_stock=10):
    # Calculate the estimated inventory levels based on TotalDemand
    product_data['EstimatedInventory'] = (product_data['TotalDemand'] * scale_factor 
                                          + np.random.normal(loc=base_stock, scale=5, size=len(product_data)))
    
    # Round and convert to integer, ensuring no negative stock levels
    product_data['EstimatedInventory'] = product_data['EstimatedInventory'].round().astype(int)
    product_data['EstimatedInventory'] = product_data['EstimatedInventory'].clip(lower=0)
    
    return product_data[['StockCode', 'TotalDemand', 'EstimatedInventory']]  # Keep TotalDemand

# Generate synthetic inventory levels with TotalDemand included
synthetic_data = generate_inventory_levels(a_items_data)

synthetic_data.head()


,StockCode,TotalDemand,EstimatedInventory
1115,22197,56427,62082
2927,84077,53751,59135
3418,85099B,47260,51999
3438,85123A,39067,42991
3216,84879,36282,39919


In [5]:
# Filter synthetic_data for A-items only

# Calculate Inventory Turnover Ratio for each A-item
synthetic_data['InventoryTurnoverRatio'] = synthetic_data['TotalDemand'] / synthetic_data['EstimatedInventory']


# Calculate the overall average inventory turnover ratio for A-items
average_turnover_ratio_a_items = synthetic_data['InventoryTurnoverRatio'].mean()

# Display the average turnover ratio for A-items
print(f"Average Inventory Turnover Ratio for A-items: {average_turnover_ratio_a_items:.2f}")

Average Inventory Turnover Ratio for A-items: 0.91



Currently, the UCI Online Retail set has an Average Inventory Turnover Ratio of less than 1, which is problematic because it indicates that inventory is turning over less than once per year, leading to high holding costs and cash tied up in unsold stock. This low turnover suggests inefficiencies, with products at risk of obsolescence and a strain on cash flow that could otherwise be invested in growth.

## How can we optimize inventory levels to minimize costs while ensuring product availability?

In this section, we develop a demand forecasting model and an inventory optimization algorithm aimed at increasing the inventory turnover ratio. By forecasting demand more accurately, we can better align stock levels with actual sales, reducing excess inventory. The optimization algorithm calculates reorder points, safety stock, and economic order quantities (EOQ) to maintain lean inventory levels, minimize holding costs, and ensure products move quickly through the inventory cycle. Together, these methods are designed to improve inventory efficiency and boost turnover.

### Demand Forecasting Model

In [6]:
# Filter the dataset for A-items based on the ABC analysis
a_items = product_demand[product_demand['ABC_Category'] == 'A']['StockCode']

# Dictionary to store forecasts for each A-item
forecasts = {}

for stockcode in a_items:
    # Prepare the product-specific time series data
    product_data = data[data['StockCode'] == stockcode]
    product_data = product_data.groupby('Invoice Date')['Quantity'].sum().reset_index()
    product_data.columns = ['ds', 'y']
    
    # Initialize and fit Prophet model
    model = Prophet(yearly_seasonality=True, weekly_seasonality=True, daily_seasonality=False)
    model.fit(product_data)
    
    # Create future dates for forecast (e.g., 52 weeks into the future)
    future_dates = model.make_future_dataframe(periods=52, freq='W')
    forecast = model.predict(future_dates)
    
    # Store the forecast in the dictionary for further analysis
    forecasts[stockcode] = forecast

# The `forecasts` dictionary now contains Prophet model results for each A-item


14:48:12 - cmdstanpy - INFO - Chain [1] start processing
14:48:12 - cmdstanpy - INFO - Chain [1] done processing
14:48:12 - cmdstanpy - INFO - Chain [1] start processing
14:48:13 - cmdstanpy - INFO - Chain [1] done processing
14:48:13 - cmdstanpy - INFO - Chain [1] start processing
14:48:13 - cmdstanpy - INFO - Chain [1] done processing
14:48:13 - cmdstanpy - INFO - Chain [1] start processing
14:48:13 - cmdstanpy - INFO - Chain [1] done processing
14:48:13 - cmdstanpy - INFO - Chain [1] start processing
14:48:13 - cmdstanpy - INFO - Chain [1] done processing
14:48:13 - cmdstanpy - INFO - Chain [1] start processing
14:48:13 - cmdstanpy - INFO - Chain [1] done processing
14:48:13 - cmdstanpy - INFO - Chain [1] start processing
14:48:13 - cmdstanpy - INFO - Chain [1] done processing
14:48:13 - cmdstanpy - INFO - Chain [1] start processing
14:48:13 - cmdstanpy - INFO - Chain [1] done processing
14:48:14 - cmdstanpy - INFO - Chain [1] start processing
14:48:14 - cmdstanpy - INFO - Chain [1]

### Inventory Optimisation Algorithm

#### 1) Economic Order Quantity (EOQ)

The EOQ is used to determine the optimal order size that minimizes total inventory costs, which include both ordering and holding costs. By ordering the ideal quantity, businesses can reduce the frequency of orders while avoiding excess stock.

The EOQ formula is given by:

$$
\text{EOQ} = \sqrt{\frac{2 \times D \times S}{H}}
$$

Where:

D = Total demand for the product (units per period)

S = Ordering cost per order

H = Holding cost per unit per period


#### 2) Safety Stock
Safety Stock is an additional quantity of inventory kept on hand to protect against uncertainties in demand and supply. It acts as a buffer to prevent stockouts during unexpected spikes in demand or delays in replenishment.

$$
\text{Safety Stock} = Z \times \sigma_d \times \sqrt{L}
$$

Where:

Z = Service level factor

**$ \sigma_d $**: The standard deviation of demand during lead time.

 L: The lead time, or the time it takes for an order to be received after it is placed.

#### 3) Reorder Point (ROP)

The **Reorder Point (ROP)** is the inventory level at which a new order should be placed to replenish stock before it runs out. This ensures that you have enough inventory on hand to cover demand during the lead time, factoring in safety stock as a buffer.

$$
\text{ROP} = \text{Lead Time Demand} + \text{Safety Stock}
$$

Where:

- **Lead Time Demand**: The expected demand during the lead time, calculated as the average demand per period multiplied by the lead time.
- **Safety Stock**: The additional inventory kept on hand to account for demand and supply variability.



In [245]:
# We assume that
ordering_cost_per_order = 50
holding_cost_per_unit_per_week = 0.5
lead_time_weeks = 2

# Dictionary to store inventory strategies for each A-item
inventory_strategies = {}

for stockcode, forecast in forecasts.items():
    # Average forecasted demand for EOQ calculation
    average_weekly_demand = forecast['yhat'].mean()
    
    # EOQ Calculation
    eoq = np.sqrt((2 * average_weekly_demand * ordering_cost_per_order) / holding_cost_per_unit_per_week)
    
    # Safety Stock calculation using forecast variability
    forecast_std_dev = (forecast['yhat_upper'] - forecast['yhat_lower']).mean() / 2
    safety_stock = forecast_std_dev * np.sqrt(lead_time_weeks)
    
    # Reorder Point (ROP) Calculation
    lead_time_demand = average_weekly_demand * lead_time_weeks
    rop = lead_time_demand + safety_stock
    
    # Store the inventory strategies
    inventory_strategies[stockcode] = {
        'EOQ': eoq,
        'Safety_Stock': safety_stock,
        'ROP': rop
    }

# The `inventory_strategies` dictionary now contains EOQ, Safety Stock, and ROP for each A-item


- **ROP check:** For each A-item, we check if inventory levels are at or below the ROP.
- **Reordering:** If inventory falls below the ROP, the system places an order for the EOQ quantity.
- **Reorder Log:** Keeps a log of all reorders, providing visibility into inventory changes and reorder points.

In [246]:
current_inventory_levels = generate_inventory_levels(product_demand).set_index('StockCode')['EstimatedInventory'].to_dict()

# Function to check inventory levels and reorder if below ROP
reorder_log = []

for stockcode, forecast in forecasts.items():
    # Access inventory strategy for this A-item
    strategy = inventory_strategies[stockcode]
    eoq = strategy['EOQ']
    rop = strategy['ROP']
    
    # Check if inventory level is at or below ROP
    if current_inventory_levels[stockcode] <= rop:
        # Place order for EOQ units
        order_quantity = eoq
        current_inventory_levels[stockcode] += order_quantity
        
        # Log the reorder
        reorder_log.append({
            'StockCode': stockcode,
            'Order_Quantity': order_quantity,
            'New_Inventory_Level': current_inventory_levels[stockcode]
        })

# Display reorder log for review
reorder_df = pd.DataFrame(reorder_log)
reorder_df


,StockCode,Order_Quantity,New_Inventory_Level
0,84826,6730.826091,21747.826091


#### Practical Tips
- **Set Up Alerts:** For each A-item, it checks if inventory levels are at or below the ROP.
- **Adjust Based on Demand Trends:** If inventory falls below the ROP, the system places an order for the EOQ quantity.

### New Average Inventory Turnover Ratio

In [247]:
# Parameters for simulation
forecast_period = 52  # Number of weeks in the forecast period

current_inventory_levels = generate_inventory_levels(product_demand).set_index('StockCode')['EstimatedInventory'].to_dict()

# Dictionary to store inventory levels over time for calculating the average
inventory_tracking = {stockcode: [] for stockcode in current_inventory_levels.keys()}

# Simulate inventory management for each week in the forecast period
for stockcode, forecast in forecasts.items():
    # Access the inventory strategy for this item
    strategy = inventory_strategies[stockcode]
    eoq = strategy['EOQ']
    rop = strategy['ROP']
    
    # Starting inventory level
    inventory_level = current_inventory_levels[stockcode]
    
    # Iterate over each week in the forecast period
    for week in range(forecast_period):
        # Predicted weekly demand for this week from the forecast
        weekly_demand = forecast['yhat'].iloc[week]
        
        # Reduce inventory by the weekly demand
        inventory_level -= weekly_demand
        
        # Log inventory level for tracking average inventory over time
        inventory_tracking[stockcode].append(max(inventory_level, 0))  # Ensure inventory doesn't go negative
        
        # After fulfilling demand, check if we need to reorder
        if inventory_level <= rop:
            # Place an order for EOQ units
            inventory_level += eoq

# Calculate the average inventory level over the period for each product
average_inventory_levels = {stockcode: np.mean(levels) for stockcode, levels in inventory_tracking.items()}

# Merge the average inventory levels with the total demand to calculate turnover ratio
turnover_data = pd.DataFrame({
    'StockCode': average_inventory_levels.keys(),
    'AverageInventoryLevel': average_inventory_levels.values()
}).merge(product_demand[['StockCode', 'TotalDemand']], on='StockCode')

# Calculate Inventory Turnover Ratio
turnover_data['InventoryTurnoverRatio'] = turnover_data['TotalDemand'] / turnover_data['AverageInventoryLevel']

# Calculate the overall average inventory turnover ratio
average_turnover_ratio = turnover_data['InventoryTurnoverRatio'].mean()

print(f"Updated Average Inventory Turnover Ratio: {average_turnover_ratio:.2f}")



Updated Average Inventory Turnover Ratio: 1.01


The increase in the average inventory turnover ratio from 0.91 to 1.01 highlights the success of our demand forecasting and inventory optimization strategy. A turnover rate above 1 means inventory is now moving faster and more in line with demand, reducing excess stock and associated holding costs. This 10% improvement not only cuts costs but also frees up cash flow and lowers the risk of obsolescence, making our inventory management leaner and more efficient.